In [10]:
import cvxpy as cp
import itertools

###############################################################################
# Helper Data Structures
###############################################################################

class UndirectedEdge:
    """
    Simple structure to hold an undirected edge. 
    We'll treat it as two directed edges internally if needed.
    """
    def __init__(self, u, v, capacity):
        self.u = min(u, v)
        self.v = max(u, v)
        self.capacity = capacity

    def __repr__(self):
        return f"({self.u}-{self.v}, cap={self.capacity})"


###############################################################################
# (A) Multi-Commodity Flow with "common per-session rate"
###############################################################################

def solve_mcf_per_session(nodes, edges, sessions):
    """
    nodes    : list of node labels
    edges    : list of UndirectedEdge objects
    sessions : list of (source, sink) pairs

    Returns (opt_value, flows, R_shared_value, per_session_R)
    """
    directed_edges = []
    for e in edges:
        directed_edges.append((e.u, e.v))
        directed_edges.append((e.v, e.u))

    k = len(sessions)

    # Flow variables: f_{(u->v), i}
    flow_vars = {}
    for (u,v) in directed_edges:
        for i in range(k):
            flow_vars[(u,v,i)] = cp.Variable(nonneg=True, name=f"flow_{u}_{v}_sess{i}")

    # Net flow for session i
    R_i = [cp.Variable(nonneg=True, name=f"R_{i}") for i in range(k)]
    # Common rate
    R_shared = cp.Variable(nonneg=True, name="R_shared")

    constraints = []

    # 1) Edge capacity: sum of flows in both directions <= capacity
    for e in edges:
        sum_flow = 0
        for i in range(k):
            sum_flow += flow_vars[(e.u, e.v, i)]
            sum_flow += flow_vars[(e.v, e.u, i)]
        constraints.append(sum_flow <= e.capacity)

    # 2) Flow conservation
    for i, (s_i, t_i) in enumerate(sessions):
        for n in nodes:
            inflow  = 0
            outflow = 0
            for (u,v) in directed_edges:
                if v == n:
                    inflow  += flow_vars[(u,v,i)]
                if u == n:
                    outflow += flow_vars[(u,v,i)]
            if n == s_i:
                # net outflow - inflow = R_i
                constraints.append(outflow - inflow == R_i[i])
            elif n == t_i:
                # net inflow - outflow = R_i
                constraints.append(inflow - outflow == R_i[i])
            else:
                # intermediate => inflow = outflow
                constraints.append(inflow == outflow)

    # 3) Enforce R_i >= R_shared
    for i in range(k):
        constraints.append(R_i[i] >= R_shared)

    # Objective: maximize R_shared
    objective = cp.Maximize(R_shared)
    prob = cp.Problem(objective, constraints)
    opt_val = prob.solve(verbose=False)

    # Extract solution
    flows_sol = {(u,v,i): flow_vars[(u,v,i)].value for (u,v,i) in flow_vars}
    R_shared_val = R_shared.value
    R_i_vals = [R_i[i].value for i in range(k)]
    return opt_val, flows_sol, R_shared_val, R_i_vals


###############################################################################
# (B) Network Coding LP with "common per-session rate"
###############################################################################

def solve_network_coding_per_session(nodes, edges, sessions, extra_cutsets=None):
    """
    nodes       : list of node labels
    edges       : list of UndirectedEdge objects
    sessions    : list of (source, sink) pairs
    extra_cutsets : list of (subset_of_sessions, capacity)
                    => we impose H(S) <= capacity
    """
    k = len(sessions)
    session_indices = range(k)

    # Define H(S) for each subset S of {0,...,k-1}
    all_subsets = []
    for r in range(k+1):
        for combo in itertools.combinations(session_indices, r):
            all_subsets.append(frozenset(combo))

    H_var = {}
    for S in all_subsets:
        # We'll include H(∅) with constraint = 0
        if len(S) == 0:
            H_var[S] = cp.Variable(nonneg=True, name="H_empty")
        else:
            name_str = "H_"+"_".join(str(x) for x in sorted(S))
            H_var[S] = cp.Variable(nonneg=True, name=name_str)

    # Single common rate
    R_shared = cp.Variable(nonneg=True, name="R_shared")

    constraints = []

    # 1) H(∅) = 0
    constraints.append(H_var[frozenset()] == 0)

    # 2) Shannon submodularity
    for S in all_subsets:
        for T in all_subsets:
            unionST = frozenset(S.union(T))
            interST = frozenset(S.intersection(T))
            constraints.append(H_var[unionST] + H_var[interST] <= H_var[S] + H_var[T])

    # 3) H({i}) >= R_shared
    for i in session_indices:
        constraints.append(H_var[frozenset([i])] >= R_shared)

    # 4) Single-edge "must cross" constraints 
    def is_separating_edge(e, s, t):
        adjacency = {n: set() for n in nodes}
        for edge_ in edges:
            if edge_ == e:
                continue
            adjacency[edge_.u].add(edge_.v)
            adjacency[edge_.v].add(edge_.u)
        visited = set()
        stack = [s]
        while stack:
            node = stack.pop()
            if node == t:
                return False
            for nbr in adjacency[node]:
                if nbr not in visited:
                    visited.add(nbr)
                    stack.append(nbr)
        return True

    for e in edges:
        must_cross = []
        for i, (s_i, t_i) in enumerate(sessions):
            if is_separating_edge(e, s_i, t_i):
                must_cross.append(i)
        if must_cross:
            S_e = frozenset(must_cross)
            constraints.append(H_var[S_e] <= e.capacity)

    # 5) Extra multi-edge cut constraints
    if extra_cutsets is not None:
        for (sess_subset, capval) in extra_cutsets:
            S_fro = frozenset(sess_subset)
            constraints.append(H_var[S_fro] <= capval)

    # Maximize R_shared
    objective = cp.Maximize(R_shared)
    prob = cp.Problem(objective, constraints)
    opt_val = prob.solve(verbose=False)

    H_sol = {S: H_var[S].value for S in all_subsets}
    R_shared_val = R_shared.value
    return opt_val, H_sol, R_shared_val


###############################################################################
# MAIN: A Small "Diamond" Network with 2 sessions
###############################################################################

def main():
    print("\n============================================")
    print(" A SIMPLE DIAMOND NETWORK (undirected)      ")
    print("============================================")

    # NODES: 0,1,2,3
    nodes = [0,1,2,3]

    # EDGES (undirected, capacity=1):
    #   (0-1), (1-3), (0-2), (2-3).
    # This forms a diamond shape:
    #     0
    #    / \
    #   1   2
    #    \ /
    #     3
    #
    edges = [
        UndirectedEdge(0,1, 1.0),
        UndirectedEdge(1,3, 1.0),
        UndirectedEdge(0,2, 1.0),
        UndirectedEdge(2,3, 1.0)
    ]

    # SESSIONS:
    #   session0:  0 -> 3
    #   session1:  1 -> 2
    sessions = [(0,3),(1,2)]

    # (A) Solve MCF (common rate)
    mcf_obj, flows, R_star_mcf, R_per_sess = solve_mcf_per_session(nodes, edges, sessions)
    print("\n--- MCF per-session solution ---")
    print(f"  Achieved common rate = {R_star_mcf:.4f}")
    for i,(s_i,t_i) in enumerate(sessions):
        print(f"    Session {i}: {s_i}->{t_i}, flow = {R_per_sess[i]:.4f}")

    # (B) Solve Network Coding
    #
    # For most small, highly connected graphs like this, single-edge constraints
    # won't reduce the capacity. So you probably won't see an advantage. 
    # We'll demonstrate "extra_cutsets" = None, since there's no known multi-edge cut
    # that restricts anything more than the single-edge constraints do.
    #
    nc_obj, H_sol, R_star_nc = solve_network_coding_per_session(
        nodes, edges, sessions,
        extra_cutsets = [
    ([0], 2.0),     # H({0}) <= 2
    ([1], 2.0),     # H({1}) <= 2
    ([0,1], 2.0)    # or maybe 2, or 3, etc. 
]

    )
    print("\n--- Network Coding per-session solution ---")
    print(f"  Achieved common rate = {R_star_nc:.4f}")
    for i in range(len(sessions)):
        val = H_sol[frozenset([i])]
        print(f"    H({{{i}}}) = {val:.4f}")

    # Summarize
    print("\nComparison:")
    print("  MCF common rate =", f"{R_star_mcf:.4f}")
    print("  NC  common rate =", f"{R_star_nc:.4f}")
    print("In this small diamond graph, they typically match (no coding advantage).")


if __name__ == "__main__":
    main()



 A SIMPLE DIAMOND NETWORK (undirected)      

--- MCF per-session solution ---
  Achieved common rate = 1.0000
    Session 0: 0->3, flow = 1.0000
    Session 1: 1->2, flow = 1.0000

--- Network Coding per-session solution ---
  Achieved common rate = 2.0000
    H({0}) = 2.0000
    H({1}) = 2.0000

Comparison:
  MCF common rate = 1.0000
  NC  common rate = 2.0000
In this small diamond graph, they typically match (no coding advantage).
